# Análise dos produtos comercializados

## Criar directoria de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = APIKEYHERE
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## China, categorias de produtos mais importantes nas trocas com os PLP

Obtem os totais agregados com nível 2 de código HS e lista os primeiros.

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


In [3]:
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    APIKEY = config['comtrade']['key']

comtrade.init(APIKEY)

### TODO

- [ ] Permitir escolher os reporter e partners
- [ ] compatibilizar com o formato de report da AICEP https://myaicep.portugalexporta.com/mercados-internacionais/cn/china?setorProduto=-1
- [ ] há problemas de duplicação de linhas nas listagem de detalhes de commodities, provavlmente por custom code, ou mot code.
  

In [43]:
import comtrade
import pandas as pd


rank_filter = 5  # número de importações mais relevantes

year_start = 1993
year_end = 2023
years = comtrade.year_range(year_start,year_end)
reporterCode = comtrade.m49_china
partnerCode = comtrade.m49_plp_list
flowCode = 'M,X'
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdCode','cmdDesc',
            'flowCode','primaryValue']



pco_top5_sorted = comtrade.top_commodities(reporterCode, partnerCode,  years, flowCode,rank_filter, pco_cols)

# save to Excel
# Prepare file name for Excel output
filename_note=f"{years.replace(',','_')}_{flowCode.replace(',','_')}"  # change to append to filename
excel_file_name = f"./reports/china_plp_top5_{filename_note}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name)

# Save data
sheet_name = "comtrade"
pco_top5_sorted.to_excel(excel_file, sheet_name=sheet_name, startrow=2)
# Adjust column width
comtrade.excel_col_autowidth(pco_top5_sorted, excel_file,sheet_name)

# format column primaryValue as currency
comtrade.excel_format_currency(pco_top5_sorted, excel_file,sheet_name, columns=['primaryValue'],width=20)
    
# Write title in first row TBD
excel_file_title = f"Trocas Comerciais China-PLP, top {rank_filter} produtos (M=Importações, X=Exportações), {years}, valores USD"
excel_file.sheets[sheet_name].write(0, 0, excel_file_title)
print("Guardado em:",excel_file_name)

excel_file.close()
# show
pd.options.display.max_colwidth=100
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 100
pco_top5_sorted.head(100)

Guardado em: ./reports/china_plp_top5_1993_1994_1995_1996_1997_1998_1999_2000_2001_2002_2003_2004_2005_2006_2007_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_2018_2019_2020_2021_2022_M_X.xlsx


cmdCode  \
reporterDesc partnerDesc flowCode refYear rank           
China        Angola      M        1993    1         27   
                                          2         84   
                                          3         85   
                                          4         48   
                                  1994    1         27   
                                          2         71   
                                          3         84   
                                          4         39   
                                          5         34   
                                  1995    1         27   
                                          2         84   
                                          3         71   
                                          4         51   
                                          5         99   
                                  1996    1         27   
                                          2         03   
                                          3         55   
                                          4         71   
                                          5         39   
                                  1997    1         27   
                                          2         39   
                                          3         03   
                                          4         90   
                                  1998    1         27   
                                          2         03   
                                          3         71   
                                          4         84   
                                  1999    1         27   
                                          2         84   
                                          3         85   
                                          4         82   
                                  2000    1         27   
                                          2         44   
                                          3         73   
                                          4         39   
                                          5         85   
                                  2001    1         27   
                                          2         44   
                                          3         03   
                                          4         25   
                                          5         88   
                                  2002    1         27   
                                          2         03   
                                          3         44   
                                          4         16   
                                          5         83   
                                  2003    1         27   
                                          2         25   
                                          3         71   
                                          4         39   
                                          5         20   
                                  2004    1         27   
                                          2         71   
                                          3         25   
                                          4         90   
                                          5         40   
                                  2005    1         27   
                                          2         71   
                                          3         73   
                                          4         44   
                                          5         03   
                                  2006    1         27   
                                          2         71   
                                          3         25   
                                          4         72   
                                          5         74   
                                  2007    1         27   
                                          2  

## Análise de variações nos produtos mais importantes

A célula seguinte produz uma lista das variações mais importantes na evolução das 
trocas comerciais entre a China e os Países de Língua Portuguesa

Por cada combinação de países e de fluxo (M=importações pela China, X=exportações da China)
por exemplo:

        China Brazil M

é apresentada uma linha por ano, por exemplo:

        1993   72,26
                -> 72 Iron and steel
                -> 26 Ores, slag and ash
        1994 ! 15,72
                -> 15 Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes
        1995 ! 15,26
        1996 ! 15,23
                -> 23 Food industries, residues and wastes thereof; prepared animal fodder
        1997 ≈ 23,15
        1998 ! 26,23
        1999 ! 26,12
                -> 12 Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder
        2000 ≈ 12,26
        2001 ≈ 26,12
        2002 ≈ 12,26
        2003 =
        2004 ≈ 26,12
        2005 =

Cada ano é seguido de um sinal que indica a mudança ocorrida em relação ao ano anterior:
* ! : mudança num dos produtos principais
* = : mesmos produtos do ano anterior, pela mesma ordem de importância
* ≈ : mesmos produtos do ano anterior, mas importância diferente


Cada vez que um novo produto aparece na série é fornecida a descrição do código respectivo.

Esta listagem pode ser produzida tendo em conta um número variáveis de produtos principais
normalmente os dois principais.

In [46]:
consider_first = 2
for reporter in pco_top5_sorted.index.get_level_values(0).unique():
    for partner in pco_top5_sorted.loc[(reporter,)].index.get_level_values(0).unique(): 
        for flow in pco_top5_sorted.loc[((reporter,partner,))].index.get_level_values(0).unique():
            print(reporter,partner, flow)
            previous_pattern = []
            all_commodities = set() # all commodities for this partner for this flow
            for year in pco_top5_sorted.loc[(reporter,partner,flow,)].index.get_level_values(0).unique():
                print(year, end=' ')
                top5_codes = pco_top5_sorted.loc[(reporter,partner,flow,year)]['cmdCode'].astype(str).values.flatten().tolist()[0:consider_first]
                top5_codes_desc = pco_top5_sorted.loc[(reporter,partner,flow,year)]['cmdDesc'].astype(str).values.flatten().tolist()[0:consider_first]
                # make set of tuples top5_codes and top5_codes_desc
                top5_codes_desc = {f"{code} {desc}" for code,desc in zip(top5_codes,top5_codes_desc)}
                new_commodities = top5_codes_desc.difference(all_commodities)
                all_commodities = all_commodities.union(top5_codes_desc)
                pattern = ','.join(top5_codes)
                if len(previous_pattern) == 0:
                    previous_pattern = top5_codes
                    print(' ', pattern)
                    for cmd in new_commodities:
                        print( "        ->",cmd)
                elif set(top5_codes) == set(previous_pattern):
                    if top5_codes == previous_pattern:
                        print('=')
                    else:
                        print('≈', pattern)
                else:
                    print('!', pattern)
                    if len(new_commodities) > 0:
                        for cmd in new_commodities:
                            print( "        ->",cmd)
                       
                previous_pattern = top5_codes
            for cmd in sorted(all_commodities):
                print( "    ",cmd)
            print(80*"-")



China Angola M
1993   27,84
        -> 84 Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof
        -> 27 Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
1994 ! 27,71
        -> 71 Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin
1995 ! 27,84
1996 ! 27,03
        -> 03 Fish and crustaceans, molluscs and other aquatic invertebrates
1997 ! 27,39
        -> 39 Plastics and articles thereof
1998 ! 27,03
1999 ! 27,84
2000 ! 27,44
        -> 44 Wood and articles of wood; wood charcoal
2001 =
2002 ! 27,03
2003 ! 27,25
        -> 25 Salt; sulphur; earths, stone; plastering materials, lime and cement
2004 ! 27,71
2005 =
2006 =
2007 =
2008 =
2009 =
2010 ! 27,25
2011 ! 27,71
2012 =
2013 =
2014 =
2015 =
2016 ! 27,44
2017 ! 27,71
2018 ! 27,44
2019 ! 27,25
2020 =
2021 =
     03 Fish and crustaceans, molluscs and ot

## Detalhe do tipo de produtos comercializados entre os PLP e a China

Em dois passos: 
* obtém as categorias de nível 2 mais importantes de cada país
* pesquisa todos as subcategorias de cada um das categorias

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [6]:

import ipywidgets as widgets
from IPython.display import display
import comtrade

rank_filter = 5  # número de importações mais relevantes
years = "2021"
partnerCode = comtrade.m49_angola # 
flowCode="M"

# select year with widget from list range(2000,2022)

years = widgets.SelectMultiple(
    options=range(2000,2022),
    value=[2021],
    description='Ano:',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
reporterCodeWidget = widgets.Dropdown(
    options=[("China", comtrade.m49_china),("Macau",comtrade.m49_macau) , ("Hong Kong", comtrade.m49_hong_kong)] + comtrade.PLP_TUPLES_REVERSE ,
    # value=[("Angola",m49_angola)],
    description='Fonte (reporter):',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
partnerCodeWidget = widgets.Dropdown(
    options=[("China", comtrade.m49_china),("Macau",comtrade.m49_macau) , ("Hong Kong", comtrade.m49_hong_kong)] + comtrade.PLP_TUPLES_REVERSE ,
    # value=[("Angola",m49_angola)],
    description='Parceiro:',
    disabled=False,
)

# select flowCode widget from list M=Importações, X=Exportações
flowCodeWidget = widgets.Dropdown(
    options=[("Importações", "M"), ("Exportações", "X")],
    value="M",
    description='Fluxo:',
    disabled=False, 
)



### Escolher ano, parceiro e fluxo (import/export) para análise 

Pode escolher-se mais que um ano.

In [7]:

display(years)
display(reporterCodeWidget)
display(partnerCodeWidget)
display(flowCodeWidget)


SelectMultiple(description='Ano:', index=(21,), options=(2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,…

Dropdown(description='Fonte (reporter):', options=(('China', 156), ('Macau', 446), ('Hong Kong', 344), ('Angol…

Dropdown(description='Parceiro:', options=(('China', 156), ('Macau', 446), ('Hong Kong', 344), ('Angola', 24),…

Dropdown(description='Fluxo:', options=(('Importações', 'M'), ('Exportações', 'X')), value='M')

Depois de escolher ano, país e fluxo executar a célula seguinte.

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [8]:
import comtrade
from comtrade import COUNTRY_CODES, HS_CODES, HS_CODES_DF, HS_CODES_L2_DF


yearsList = ",".join(list(map(str,years.value)))
partnerCode = partnerCodeWidget.value
reporterCode = reporterCodeWidget.value


# decode country
partner_name = comtrade.COUNTRY_CODES.get(partnerCode)
reporter_name = comtrade.COUNTRY_CODES.get(reporterCode)

flow = flowCodeWidget.value
print(reporter_name,yearsList,flow,partner_name,)

pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']

pco_cols_detail = ['reporterDesc','partnerDesc','refYear','cmdCodeAG2','cmdCode','cmdDesc',
            'flowCode','primaryValue', 'isAggregate']

# first we collect the top commodity level 2 codes            
df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     cmdCode="AG2",
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     period=yearsList,
                     timeout=30,
                     echo_url=True
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] <= rank_filter]
# get the countries
countries = pco_top5.partnerDesc.unique()
country_cmd_top5_codes = dict()

# get the detailed commodity codes for the top of each country
for country in countries:
    l2_codes = pco_top5[pco_top5.partnerDesc == country]['cmdCode'].unique()
    print(country,l2_codes)
    hs_details = []
    for l2_code in l2_codes:
        l2_sub_codes = list(HS_CODES_DF[HS_CODES_DF.hscode.str.startswith(l2_code)]['hscode'])
        hs_details = hs_details + l2_sub_codes
    # print(hs_details)
    country_cmd_top5_codes[country] = hs_details.copy()

# now we fetch the detail
countryDesc = COUNTRY_CODES[partnerCode]
country_cmd_codes = ",".join(country_cmd_top5_codes[countryDesc])
print(country_cmd_codes)

df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     #cmdCode=country_cmd_codes, # sometimes the detail commodity codes generates errors
                     cmdCode=None, # This gives all the commodities for the country and year 
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     partner2Code=0,
                     period=yearsList,
                     echo_url=True,
                     timeout=60
                     )
df['cmdCodeAG2'] = df.cmdCode.str[0:2]
# filter the detail commodity codes by the top 5
df = df[df.cmdCode.isin(country_cmd_top5_codes[countryDesc])]

df['subtotalAG2'] = df.groupby(['partnerCode','refYear','flowCode','cmdCodeAG2'])["primaryValue"].transform('sum')

df[pco_cols_detail].sort_values('primaryValue',ascending=False).to_excel(f"./reports/product_detail_{reporter_name}_{yearsList}_{flow}_{partner_name}.xlsx")

pd.options.display.max_rows = 500
df.sort_values(['partnerDesc','refYear','flowCode','subtotalAG2','cmdCodeAG2','primaryValue'],ascending=[True,True,True,False,True,False])[pco_cols_detail].head(500)

China 2021 M Angola
https://comtradeapi.un.org/data/v1/get//C/A/HS?reporterCode=156&period=2021&partnerCode=24&partner2Code=0&cmdCode=AG2&flowCode=M&customsCode=C00&subscription-key=HIDDEN
Angola ['27' '25' '26' '74' '76']
27,2701,270111,270112,270119,270120,2702,270210,270220,2703,270300,2704,270400,2705,270500,2706,270600,2707,270710,270720,270730,270740,270750,270791,270799,2708,270810,270820,2709,270900,2710,271012,271019,271020,271091,271099,2711,271111,271112,271113,271114,271119,271121,271129,2712,271210,271220,271290,2713,271311,271312,271320,271390,2714,271410,271490,2715,271500,2716,271600,25,2501,250100,2502,250200,2503,250300,2504,250410,250490,2505,250510,250590,2506,250610,250620,2507,250700,2508,250810,250830,250840,250850,250860,250870,2509,250900,2510,251010,251020,2511,251110,251120,2512,251200,2513,251310,251320,2514,251400,2515,251511,251512,251520,2516,251611,251612,251620,251690,2517,251710,251720,251730,251741,251749,2518,251810,251820,2519,251910,251990,2520,252

,reporterDesc,partnerDesc,refYear,cmdCodeAG2,cmdCode,cmdDesc,flowCode,primaryValue,isAggregate
26,China,Angola,2021,27,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral w...",M,"20,823,873,963.00",True
27,China,Angola,2021,27,2709,Petroleum oils and oils obtained from bituminous minerals; crude,M,"19,855,580,399.00",True
28,China,Angola,2021,27,270900,"Oils; petroleum oils and oils obtained from bituminous minerals, crude",M,"19,855,580,399.00",False
29,China,Angola,2021,27,2711,Petroleum gases and other gaseous hydrocarbons,M,"767,778,092.00",True
31,China,Angola,2021,27,271112,"Petroleum gases and other gaseous hydrocarbons; liquefied, propane",M,"423,920,134.00",False
30,China,Angola,2021,27,271111,"Petroleum gases and other gaseous hydrocarbons; liquefied, natural gas",M,"305,044,314.00",False
33,China,Angola,2021,27,2715,"Bituminous mixtures based on natural asphalt; on natural bitumen, on petroleum bitumen, on miner...",M,"200,515,472.00",True
34,China,Angola,2021,27,271500,"Bituminous mixtures based on natural asphalt, on natural bitumen, on petroleum bitumen, on miner...",M,"200,515,472.00",False
32,China,Angola,2021,27,271113,"Petroleum gases and other gaseous hydrocarbons; liquefied, butanes",M,"38,813,644.00",False
3,China,Angola,2021,25,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",M,"30,420,529.00",True
